<a href="https://colab.research.google.com/github/Osondu-ifunanya/Surface-water-mapping-using-ML-on-NDWI-enhanced-imagery/blob/main/SurfaceWaterMapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from skimage.util import random_noise

# -----------------------------
# 1. Generate synthetic imagery
# -----------------------------
np.random.seed(42)

n_images = 500            # number of image patches (samples)
img_h, img_w = 32, 32     # patch size
bands = ['Green', 'NIR']  # we simulate two bands to compute NDWI

# Helper to create synthetic patch
def make_patch(is_water, h=img_h, w=img_w):
    """
    Create a synthetic patch:
      - water: low Green, high NIR? (for NDWI we often use (Green - NIR)/(Green + NIR))
      - non-water: higher Green relative to NIR or textured
    We'll simulate realistic contrasts and add noise.
    """
    if is_water:
        # water: generally low reflectance in NIR, moderate in green
        green = np.random.uniform(0.06, 0.25, size=(h, w))
        nir   = np.random.uniform(0.02, 0.12, size=(h, w))
        # add calm-water specular spots
        mask_spots = (np.random.rand(h, w) > 0.995)
        green[mask_spots] += np.random.uniform(0.1, 0.3)
    else:
        # land: vegetation/soil -> higher NIR, variable green
        green = np.random.uniform(0.05, 0.6, size=(h, w))
        nir   = np.random.uniform(0.1, 0.8, size=(h, w))
        # simulate textures: fields, built-up
        if np.random.rand() < 0.5:
            green += np.random.normal(0, 0.02, size=(h, w))
            nir   += np.random.normal(0, 0.03, size=(h, w))

    # clip and add small Gaussian noise
    green = np.clip(random_noise(green, mode='gaussian', var=1e-4), 0, 1)
    nir   = np.clip(random_noise(nir,   mode='gaussian', var=1e-4), 0, 1)
    return green.astype(np.float32), nir.astype(np.float32)

# Build dataset
patches_green = []
patches_nir = []
labels = []

for i in range(n_images):
    # randomly assign water prevalence: allow some mixed patches
    is_water = np.random.rand() < 0.4  # 40% water patches on average
    g, n = make_patch(is_water)
    # optionally create mixed patches (partial water)
    if np.random.rand() < 0.2:
        # mix with opposite class small region
        g2, n2 = make_patch(not is_water)
        # overlay a rectangle of g2 onto center
        r_h, r_w = int(img_h*0.3), int(img_w*0.3)
        y0 = (img_h - r_h)//2; x0 = (img_w - r_w)//2
        g[y0:y0+r_h, x0:x0+r_w] = g2[y0:y0+r_h, x0:x0+r_w]
        n[y0:y0+r_h, x0:x0+r_w] = n2[y0:y0+r_h, x0:x0+r_w]
        # label as water if majority pixels are water according to simple NDWI rule
        # we'll compute true label below
    patches_green.append(g)
    patches_nir.append(n)

# -----------------------------
# 2. Compute NDWI and derive labels
# -----------------------------
def compute_ndwi(g, n):
    # NDWI = (Green - NIR) / (Green + NIR)
    denom = (g + n)
    # avoid division by zero
    denom = np.where(denom == 0, 1e-6, denom)
    return (g - n) / denom

ndwi_patches = [compute_ndwi(g, n) for g, n in zip(patches_green, patches_nir)]

# Derive binary label per patch: water if > threshold fraction of pixels have NDWI > 0
ndwi_threshold = 0.0
water_fraction_threshold = 0.3  # if 30%+ pixels indicate water -> patch = water

for ndwi in ndwi_patches:
    water_pixels = (ndwi > ndwi_threshold).sum()
    frac = water_pixels / (img_h * img_w)
    labels.append(1 if frac >= water_fraction_threshold else 0)

# Build feature vectors: use NDWI statistics + spectral band stats (can be expanded)
X_features = []
for g, n, nd in zip(patches_green, patches_nir, ndwi_patches):
    feat = [
        nd.mean(),
        nd.std(),
        np.percentile(nd.flatten(), 75),
        np.percentile(nd.flatten(), 25),
        g.mean(),
        g.std(),
        n.mean(),
        n.std(),
    ]
    X_features.append(feat)

X = np.array(X_features)
y = np.array(labels)

# -----------------------------
# 3. Train/test split and model
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

clf = RandomForestClassifier(n_estimators=200, random_state=42, min_samples_leaf=3)
clf.fit(X_train, y_train)

# -----------------------------
# 4. Evaluation
# -----------------------------
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# -----------------------------
# 5. Visualize some examples (NDWI map + predicted label)
# -----------------------------
def show_patch(idx):
    g = patches_green[idx]
    n = patches_nir[idx]
    nd = compute_ndwi(g, n)
    fig, axes = plt.subplots(1, 3, figsize=(10, 3))
    axes[0].imshow(g, cmap='Greens'); axes[0].set_title('Green')
    axes[1].imshow(n, cmap='inferno'); axes[1].set_title('NIR')
    im = axes[2].imshow(nd, cmap='BrBG', vmin=-1, vmax=1)
    axes[2].set_title(f'NDWI\nTrue={y[idx]}')
    fig.colorbar(im, ax=axes[2], fraction=0.046)
    plt.suptitle(f"Sample {idx} - Predicted (by model on patch stats): {clf.predict([X_features[idx]])[0]}")
    plt.tight_layout()
    plt.show()

# show a few examples (some water, some non-water)
for idx in [0, 5, 12]:
    show_patch(idx)

# -----------------------------
# 6. Feature importance
# -----------------------------
feat_names = ['NDWI_mean','NDWI_std','NDWI_p75','NDWI_p25','G_mean','G_std','NIR_mean','NIR_std']
importances = clf.feature_importances_
fi_df = pd.DataFrame({'feature': feat_names, 'importance': importances}).sort_values('importance', ascending=False)
print("\nFeature importances:\n", fi_df)

# -----------------------------
# 7. Export dataset (per-patch) to Excel for inspection
# -----------------------------
df_export = pd.DataFrame(X, columns=feat_names)
df_export['Label'] = y
# add a few raw stats for context
df_export['WaterPixelFraction'] = [(nd > ndwi_threshold).sum()/(img_h*img_w) for nd in ndwi_patches]
excel_path = "synthetic_surface_water_dataset.xlsx"
df_export.to_excel(excel_path, index=False)
print(f"\nExported dataset to: {excel_path}")

# -----------------------------
# 8. Optional: Pixel-level classifier (demo)
# Build a pixel-wise dataset (NDWI per pixel) to train a pixel classifier
# -----------------------------
# Create pixel-level features and labels (this demonstrates more granular mapping)
pixel_feats = []
pixel_labels = []
for nd, g, n in zip(ndwi_patches, patches_green, patches_nir):
    # Each pixel described by [NDWI, Green, NIR]
    pix_feat = np.stack([nd.flatten(), g.flatten(), n.flatten()], axis=1)
    # label pixel water if NDWI > threshold
    pix_label = (nd.flatten() > ndwi_threshold).astype(int)
    pixel_feats.append(pix_feat)
    pixel_labels.append(pix_label)

pixel_X = np.vstack(pixel_feats)
pixel_y = np.hstack(pixel_labels)

# sample down for speed
sample_idx = np.random.choice(len(pixel_y), size=min(20000, len(pixel_y)), replace=False)
pixel_X_s = pixel_X[sample_idx]
pixel_y_s = pixel_y[sample_idx]

px_X_train, px_X_test, px_y_train, px_y_test = train_test_split(pixel_X_s, pixel_y_s, test_size=0.3, random_state=42, stratify=pixel_y_s)
px_clf = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_leaf=5)
px_clf.fit(px_X_train, px_y_train)
px_pred = px_clf.predict(px_X_test)
print("\nPixel-level classifier accuracy:", accuracy_score(px_y_test, px_pred))

